In [1]:
!pip install gradio scikit-learn pandas matplotlib seaborn nltk xgboost


In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('/content/airlines_flights_data.csv')
df.head()


,index,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

df = pd.read_csv('/content/airlines_flights_data.csv')

features = ['airline', 'source_city', 'destination_city', 'departure_time', 'stops', 'class', 'days_left']
target = 'price'

X = df[features].copy()
y = df[target]

le_dict = {}
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    le_dict[col] = le

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [12]:
joblib.dump(model, 'price_model.pkl')
joblib.dump(le_dict, 'price_encoders.pkl')


['price_encoders.pkl']

In [13]:
def predict_price(airline, source_city, destination_city, departure_time, stops, travel_class, days_left):
    model = joblib.load('price_model.pkl')
    encoders = joblib.load('price_encoders.pkl')

    data = {
        'airline': encoders['airline'].transform([airline])[0],
        'source_city': encoders['source_city'].transform([source_city])[0],
        'destination_city': encoders['destination_city'].transform([destination_city])[0],
        'departure_time': encoders['departure_time'].transform([departure_time])[0],
        'stops': encoders['stops'].transform([stops])[0],
        'class': encoders['class'].transform([travel_class])[0],
        'days_left': int(days_left)
    }

    input_df = pd.DataFrame([data])
    predicted_price = model.predict(input_df)[0]
    return f"Predicted Ticket Price: ₹{round(predicted_price, 2)}"


In [14]:
!pip install gradio --quiet


In [22]:
import gradio as gr
import joblib
import pandas as pd

model = joblib.load('price_model.pkl')
encoders = joblib.load('price_encoders.pkl')

def predict_price(airline, source_city, destination_city, departure_time, stops, travel_class, days_left):
    try:
        data = {
            'airline': encoders['airline'].transform([airline])[0],
            'source_city': encoders['source_city'].transform([source_city])[0],
            'destination_city': encoders['destination_city'].transform([destination_city])[0],
            'departure_time': encoders['departure_time'].transform([departure_time])[0],
            'stops': encoders['stops'].transform([stops])[0],
            'class': encoders['class'].transform([travel_class])[0],
            'days_left': int(days_left)
        }

        input_df = pd.DataFrame([data])
        predicted_price = model.predict(input_df)[0]
        return f"✈️ Predicted Ticket Price: ₹{round(predicted_price, 2)}"

    except Exception as e:
        return f"Error: {str(e)}"

airlines = encoders['airline'].classes_.tolist()
sources = encoders['source_city'].classes_.tolist()
destinations = encoders['destination_city'].classes_.tolist()
dep_times = encoders['departure_time'].classes_.tolist()
stops = encoders['stops'].classes_.tolist()
classes = encoders['class'].classes_.tolist()

with gr.Blocks(css="""
body {
    background: url('https://www.transparenttextures.com/patterns/45-degree-fabric-light.png'), linear-gradient(135deg, #1e3a8a, #3b82f6);
    color: #ffffff;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
.gradio-container {
    max-width: 1000px;
    margin: 20px auto;
    padding: 20px;
    background: rgba(255, 255, 255, 0.95);
    border-radius: 15px;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.2);
}
h1 {
    font-size: 2.2em;
    font-weight: 700;
    color: #1e3a8a;
    text-align: center;
    margin-bottom: 10px;
    text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.1);
}
h3 {
    font-size: 1.5em;
    color: #4b5563;
    text-align: center;
    margin-bottom: 20px;
}
.gr-row {
    display: flex;
    gap: 15px;
    flex-wrap: wrap;
}
.gr-dropdown, .gr-slider {
    background: #f9fafb;
    border: 1px solid #d1d5db;
    border-radius: 8px;
    padding: 10px;
}
button {
    background: linear-gradient(to right, #3b82f6, #1e3a8a) !important;
    color: white !important;
    font-size: 1.1em !important;
    font-weight: 600;
    border-radius: 8px !important;
    padding: 12px 24px !important;
    transition: all 0.3s ease;
}
button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.15);
}
.output-textbox textarea {
    color: #1e3a8a;
    font-size: 1.4em;
    font-weight: 600;
    background: #eff6ff;
    border: 2px solid #3b82f6;
    border-radius: 10px;
    padding: 15px;
    text-align: center;
}
.theme-toggle {
    position: absolute;
    top: 20px;
    right: 20px;
    cursor: pointer;
}
@media (max-width: 600px) {
    .gr-row {
        flex-direction: column;
    }
}
""") as app:

    gr.HTML("""
    <h1>Flight Price Dashboard</h1>
    <h3>Estimate your flight fare with real-time predictions</h3>
    <input type='checkbox' class='theme-toggle' onchange='document.body.classList.toggle("dark-mode")' title='Toggle Theme'/>
    """)

    with gr.Row():
        airline_input = gr.Dropdown(choices=airlines, label="Airline")
        source_input = gr.Dropdown(choices=sources, label="Source City")
        destination_input = gr.Dropdown(choices=destinations, label="Destination City")

    with gr.Row():
        dep_input = gr.Dropdown(choices=dep_times, label="Departure Time")
        stop_input = gr.Dropdown(choices=stops, label="Stops")
        class_input = gr.Dropdown(choices=classes, label="Class")

    days_input = gr.Slider(1, 60, value=10, label="Days Left Until Departure")

    predict_btn = gr.Button("Predict Price")
    output = gr.Textbox(label="Predicted Price", elem_classes=["output-textbox"])

    predict_btn.click(fn=predict_price,
                      inputs=[airline_input, source_input, destination_input, dep_input, stop_input, class_input, days_input],
                      outputs=output)

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ceb8a7123e5f02a4dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
